# Greek Derby RAG System - Olympiakos vs Panathinaikos
## RAG System for Greek Football Derby Analysis using Gazzetta.gr


## Loading Environment Variables


In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify API keys are loaded
print(f"OpenAI API Key loaded: {bool(os.getenv('OPENAI_API_KEY'))}")
print(f"Pinecone API Key loaded: {bool(os.getenv('PINECONE_API_KEY'))}")
print(f"Pinecone Index Name: {os.getenv('PINECONE_INDEX_NAME')}")


OpenAI API Key loaded: True
Pinecone API Key loaded: True
Pinecone Index Name: rag


## Selecting Chat Model for Greek Language


In [2]:
from langchain.chat_models import init_chat_model

# Initialize GPT-4o-mini with Greek language support
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# Test Greek language capability
test_response = llm.invoke("Γεια σας! Μπορείτε να μιλήσετε ελληνικά;")
print(f"Greek Language Test: {test_response.content}")


Greek Language Test: Γεια σας! Ναι, μπορώ να μιλήσω ελληνικά. Πώς μπορώ να σας βοηθήσω σήμερα;


## Selecting Embeddings Model for Greek Text


In [3]:
from langchain_openai import OpenAIEmbeddings

# OpenAI Embedding Models optimized for multilingual content including Greek:
# - text-embedding-3-small: 1536 dimensions (good for Greek)
# - text-embedding-3-large: 3072 dimensions (best for Greek)
# - text-embedding-3-small with dimensions=1024: 1024 dimensions (balanced)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024  # Good balance for Greek text
)

# Test Greek text embedding
test_text = "Ολυμπιακός vs Παναθηναϊκός - το μεγάλο ντέρμπι της Ελλάδας"
test_embedding = embeddings.embed_query(test_text)
print(f"Greek text embedding dimension: {len(test_embedding)}")


Greek text embedding dimension: 1024


## Setting up Vector Store for Greek Content


In [4]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
index = pc.Index("greekderby")

# Create vector store for Greek content
vector_store = PineconeVectorStore(embedding=embeddings, index=index)

print(f"Vector store initialized for Greek content")
print(f"Index stats: {index.describe_index_stats()}")


c:\Users\PX\anaconda3\envs\music-chatbot\lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore
Index host ignored when initializing with index object.


Vector store initialized for Greek content
Index stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 160}},
 'total_vector_count': 160,
 'vector_type': 'dense'}


## Loading Greek Football Content from Gazzetta.gr


In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from urllib.parse import urljoin, urlparse
import time
import requests

# Set user agent to avoid blocking
os.environ['USER_AGENT'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'

# URLs to scrape from Gazzetta.gr related to Olympiakos and Panathinaikos
greek_derby_urls = [
    "https://www.gazzetta.gr/football/superleague/olympiakos",
    "https://www.gazzetta.gr/football/superleague/panathinaikos",
    "https://www.gazzetta.gr/football/superleague"
]

print("Loading Greek football content from Gazzetta.gr...")

# Load content from multiple URLs with better selectors
all_docs = []
for url in greek_derby_urls:
    try:
        print(f"Loading: {url}")
        
        # First try with broader selectors to get more content
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("article-content", "article-title", "article-body", "content", "post-content", 
                           "entry-content", "post-body", "article-text", "main-content", "story-content",
                           "article", "post", "content-area", "main", "body")
                )
            ),
        )
        docs = loader.load()
        
        # If no content found, try without any class filtering
        if not docs or all(len(doc.page_content.strip()) < 100 for doc in docs):
            print(f"  No content found with selectors, trying without filtering...")
            loader_fallback = WebBaseLoader(web_paths=(url,))
            docs = loader_fallback.load()
        
        # Filter out very short documents
        valid_docs = [doc for doc in docs if len(doc.page_content.strip()) > 50]
        all_docs.extend(valid_docs)
        
        print(f"  Found {len(valid_docs)} valid documents from {url}")
        time.sleep(1)  # Be respectful to the server
        
    except Exception as e:
        print(f"Error loading {url}: {e}")
        continue

print(f"Loaded {len(all_docs)} documents from Gazzetta.gr")

# If still no content, try a different approach with requests
if len(all_docs) == 0 or all(len(doc.page_content.strip()) < 100 for doc in all_docs):
    print("\nTrying alternative approach with requests...")
    try:
        response = requests.get("https://www.gazzetta.gr/football/superleague", 
                              headers={'User-Agent': os.environ['USER_AGENT']})
        if response.status_code == 200:
            from langchain_core.documents import Document
            # Create a document from the raw HTML content
            fallback_doc = Document(
                page_content=response.text,
                metadata={"source": "https://www.gazzetta.gr/football/superleague", "method": "requests"}
            )
            all_docs.append(fallback_doc)
            print("Added fallback document from requests")
    except Exception as e:
        print(f"Fallback approach also failed: {e}")


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading Greek football content from Gazzetta.gr...
Loading: https://www.gazzetta.gr/football/superleague/olympiakos
  No content found with selectors, trying without filtering...
  Found 1 valid documents from https://www.gazzetta.gr/football/superleague/olympiakos
Loading: https://www.gazzetta.gr/football/superleague/panathinaikos
  No content found with selectors, trying without filtering...
  Found 1 valid documents from https://www.gazzetta.gr/football/superleague/panathinaikos
Loading: https://www.gazzetta.gr/football/superleague
  No content found with selectors, trying without filtering...
  Found 1 valid documents from https://www.gazzetta.gr/football/superleague
Loaded 3 documents from Gazzetta.gr


## Splitting Greek Documents


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Debug: Check what's actually in the loaded documents
print("Debugging loaded documents:")
for i, doc in enumerate(all_docs):
    print(f"\nDocument {i+1}:")
    print(f"  Length: {len(doc.page_content)} characters")
    print(f"  Content preview: {doc.page_content[:200]}...")
    print(f"  Metadata: {doc.metadata}")

# Try with smaller chunk size to see if we can get any chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Reduced from 1000
    chunk_overlap=100,  # Reduced from 200
    separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]  # Greek-friendly separators
)

all_splits = text_splitter.split_documents(all_docs)
print(f"\nCreated {len(all_splits)} text chunks from Greek content")

# If still 0 chunks, try even smaller
if len(all_splits) == 0:
    print("\nTrying with even smaller chunks...")
    text_splitter_small = RecursiveCharacterTextSplitter(
        chunk_size=100,  # Very small chunks
        chunk_overlap=20,
        separators=["\n", ". ", "! ", "? ", " ", ""]
    )
    all_splits = text_splitter_small.split_documents(all_docs)
    print(f"Created {len(all_splits)} text chunks with small size")

# Show sample chunks if any were created
if all_splits:
    print(f"\nSample chunk content:")
    print(f"Length: {len(all_splits[0].page_content)} characters")
    print(f"Content: {all_splits[0].page_content[:300]}...")


Debugging loaded documents:

Document 1:
  Length: 19839 characters
  Content preview: Ποδόσφαιρο Ολυμπιακός Νέα - Ειδήσεις - Αγώνες | gazzetta.gr
   Παράκαμψη προς το κυρίως περιεχόμενο       Slogun:Αφού η Κηφισιά δήλωσε έδρα το Αγρίνιο, ο Παναιτωλικός με τέτοια αποτελέσματα ψάχνει γήπ...
  Metadata: {'source': 'https://www.gazzetta.gr/football/superleague/olympiakos', 'title': 'Ποδόσφαιρο Ολυμπιακός Νέα - Ειδήσεις - Αγώνες | gazzetta.gr', 'description': 'Ανακαλύψτε τα τελευταία νέα Ποδόσφαιρο Ολυμπιακός στο gazzetta.gr. Ενημερωθείτε πρώτοι για όλες τις εξελίξεις Ποδόσφαιρο Ολυμπιακός και μείνετε συντονισμένοι!', 'language': 'el'}

Document 2:
  Length: 18613 characters
  Content preview: Ποδόσφαιρο Παναθηναϊκός Νέα - Ειδήσεις - Αγώνες | gazzetta.gr
   Παράκαμψη προς το κυρίως περιεχόμενο       Slogun:Αφού η Κηφισιά δήλωσε έδρα το Αγρίνιο, ο Παναιτωλικός με τέτοια αποτελέσματα ψάχνει γ...
  Metadata: {'source': 'https://www.gazzetta.gr/football/superleague/panathinaikos', 'title': 'Πο

## Storing Greek Content in Vector Database


In [7]:
# Clear existing content and add new Greek content
print("Storing Greek football content in vector database...")

# Add documents to vector store
_ = vector_store.add_documents(documents=all_splits)

# Check index stats
stats = index.describe_index_stats()
print(f"Vector database stats: {stats}")
print(f"Total vectors: {stats['total_vector_count']}")


Storing Greek football content in vector database...
Vector database stats: {'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 193}},
 'total_vector_count': 193,
 'vector_type': 'dense'}
Total vectors: 193


## Creating Greek Language RAG Prompt


In [8]:
from langchain.prompts import PromptTemplate

# Custom Greek language prompt for football derby questions
greek_prompt_template = """
Είστε ένας βοηθός για ερωτήσεις σχετικά με το ελληνικό ποδόσφαιρο και το ντέρμπι Ολυμπιακός-Παναθηναϊκός.
Χρησιμοποιήστε τα παρακάτω κομμάτια πληροφοριών για να απαντήσετε στην ερώτηση.
Αν δεν γνωρίζετε την απάντηση, απλά πείτε ότι δεν γνωρίζετε.
Χρησιμοποιήστε μέχρι τρεις προτάσεις και κρατήστε την απάντηση συνοπτική.
Απαντήστε στα ελληνικά.

Ερώτηση: {question}
Περιεχόμενο: {context}
Απάντηση:"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=greek_prompt_template
)

print("Greek RAG prompt created successfully")
print(f"Prompt template: {prompt.template[:100]}...")


Greek RAG prompt created successfully
Prompt template: 
Είστε ένας βοηθός για ερωτήσεις σχετικά με το ελληνικό ποδόσφαιρο και το ντέρμπι Ολυμπιακός-Παναθην...


## Setting up Greek RAG Functions


In [9]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

# Define state for Greek RAG application
class GreekDerbyState(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps for Greek content
def retrieve_greek_content(state: GreekDerbyState):
    """Retrieve relevant Greek football content based on question"""
    retrieved_docs = vector_store.similarity_search(
        state["question"], 
        k=4  # Get top 4 most relevant documents
    )
    return {"context": retrieved_docs}

def generate_greek_answer(state: GreekDerbyState):
    """Generate answer in Greek based on retrieved context"""
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

print("Greek RAG functions defined successfully")


Greek RAG functions defined successfully


## Building the Greek RAG LangGraph


In [10]:
from langgraph.graph import START, StateGraph

# Build the Greek RAG graph
graph_builder = StateGraph(GreekDerbyState)
graph_builder.add_sequence([retrieve_greek_content, generate_greek_answer])
graph_builder.add_edge(START, "retrieve_greek_content")

# Compile the graph
greek_derby_graph = graph_builder.compile()

print("Greek Derby RAG graph compiled successfully")
print(f"Graph nodes: {list(greek_derby_graph.get_graph().nodes.keys())}")


Greek Derby RAG graph compiled successfully
Graph nodes: ['__start__', 'retrieve_greek_content', 'generate_greek_answer', '__end__']


## Testing the Greek Derby RAG System


In [11]:
# Test questions about Olympiakos vs Panathinaikos derby
test_questions = [
    "Ποια είναι η ιστορία του ντέρμπι Ολυμπιακός-Παναθηναϊκός;",
    "Ποιος έχει κερδίσει περισσότερες φορές το ντέρμπι;",
    "Ποια είναι τα πιο αξέχαστα γκολ στο ντέρμπι;",
    "Ποιοι είναι οι κορυφαίοι παίκτες που έχουν αγωνιστεί στο ντέρμπι;",
    "Ποια είναι η σημασία του ντέρμπι για τους φιλάθλους;"
]

print("Testing Greek Derby RAG System...")
print("=" * 50)

for i, question in enumerate(test_questions, 1):
    print(f"\nΕρώτηση {i}: {question}")
    print("-" * 30)
    
    try:
        response = greek_derby_graph.invoke({"question": question})
        print(f"Απάντηση: {response['answer']}")
    except Exception as e:
        print(f"Σφάλμα: {e}")
    
    print()


Testing Greek Derby RAG System...

Ερώτηση 1: Ποια είναι η ιστορία του ντέρμπι Ολυμπιακός-Παναθηναϊκός;
------------------------------
Απάντηση: Η ιστορία του ντέρμπι Ολυμπιακός-Παναθηναϊκός είναι γεμάτη εντάσεις και ανταγωνισμό, καθώς πρόκειται για μία από τις πιο ιστορικές αντιπαραθέσεις στο ελληνικό ποδόσφαιρο. Οι δύο ομάδες, γνωστές και ως "αιώνιοι αντίπαλοι", έχουν συναντηθεί πολλές φορές και οι αγώνες τους συχνά κρίνουν τίτλους και φιλοδοξίες. Το ντέρμπι συνήθως συνοδεύεται από πάθος και είναι γεμάτο ειδικές συνθήκες, όπως αλλαγές προπονητών και άλλες συγκυρίες που επηρεάζουν την ψυχολογία των ομάδων.


Ερώτηση 2: Ποιος έχει κερδίσει περισσότερες φορές το ντέρμπι;
------------------------------
Απάντηση: Δεν γνωρίζω.


Ερώτηση 3: Ποια είναι τα πιο αξέχαστα γκολ στο ντέρμπι;
------------------------------
Απάντηση: Δεν γνωρίζω συγκεκριμένα ποια είναι τα πιο αξέχαστα γκολ στο ντέρμπι Ολυμπιακού-Παναθηναϊκού.


Ερώτηση 4: Ποιοι είναι οι κορυφαίοι παίκτες που έχουν αγωνιστεί στο ντέρ

## Interactive Greek Derby Chat


In [12]:
def ask_greek_derby_question(question: str):
    """Interactive function to ask questions about the Greek derby"""
    print(f"Ερώτηση: {question}")
    print("Αναζήτηση πληροφοριών...")
    
    try:
        response = greek_derby_graph.invoke({"question": question})
        print(f"\nΑπάντηση: {response['answer']}")
        
        # Show sources
        print("\nΠηγές:")
        for i, doc in enumerate(response['context'], 1):
            print(f"{i}. {doc.page_content[:100]}...")
            
    except Exception as e:
        print(f"Σφάλμα: {e}")


# Uncomment to test interactive mode
ask_greek_derby_question("Που γίνεται το ντέρμπι Ολυμπιακός-Παναθηναϊκός;")


Ερώτηση: Που γίνεται το ντέρμπι Ολυμπιακός-Παναθηναϊκός;
Αναζήτηση πληροφοριών...

Απάντηση: Το ντέρμπι Ολυμπιακός-Παναθηναϊκός γίνεται συνήθως στο γήπεδο του Ολυμπιακού, το Γεώργιος Καραϊσκάκης, ή στο γήπεδο του Παναθηναϊκού, το Απόστολος Νικολαΐδης, ανάλογα με την έδρα της κάθε ομάδας.

Πηγές:
1. Ολυμπιακός: Ο δρόμος για το ντέρμπι είναι ένας και ειδικά οι «μπαρουτοκαπνισμένοι» τον ξέρουν καλά  ...
2. Ολυμπιακός: Ο δρόμος για το ντέρμπι είναι ένας και ειδικά οι «μπαρουτοκαπνισμένοι» τον ξέρουν καλά  ...
3. Ολυμπιακός: Ο δρόμος για το ντέρμπι είναι ένας και ειδικά οι «μπαρουτοκαπνισμένοι» τον ξέρουν καλά  ...
4. Ολυμπιακός: Ο δρόμος για το ντέρμπι είναι ένας και ειδικά οι «μπαρουτοκαπνισμένοι» τον ξέρουν καλά  ...


## Performance Metrics and Monitoring


In [13]:
import time
from typing import Dict, Any

def benchmark_greek_rag(questions: list) -> Dict[str, Any]:
    """Benchmark the Greek RAG system performance"""
    results = {
        'total_questions': len(questions),
        'successful_answers': 0,
        'failed_answers': 0,
        'average_response_time': 0,
        'total_time': 0
    }
    
    start_time = time.time()
    
    for question in questions:
        question_start = time.time()
        try:
            response = greek_derby_graph.invoke({"question": question})
            if response['answer']:
                results['successful_answers'] += 1
            else:
                results['failed_answers'] += 1
        except Exception as e:
            results['failed_answers'] += 1
            print(f"Error with question '{question}': {e}")
        
        question_time = time.time() - question_start
        results['total_time'] += question_time
    
    results['average_response_time'] = results['total_time'] / len(questions)
    
    return results

# Run benchmark
benchmark_questions = [
    "Ποια είναι η ιστορία του ντέρμπι;",
    "Ποιος έχει κερδίσει περισσότερες φορές;",
    "Ποια είναι τα πιο αξέχαστα γκολ;"
]

print("Running Greek RAG benchmark...")
benchmark_results = benchmark_greek_rag(benchmark_questions)

print("\nBenchmark Results:")
print(f"Total Questions: {benchmark_results['total_questions']}")
print(f"Successful Answers: {benchmark_results['successful_answers']}")
print(f"Failed Answers: {benchmark_results['failed_answers']}")
print(f"Average Response Time: {benchmark_results['average_response_time']:.2f} seconds")
print(f"Success Rate: {(benchmark_results['successful_answers'] / benchmark_results['total_questions']) * 100:.1f}%")


Running Greek RAG benchmark...

Benchmark Results:
Total Questions: 3
Successful Answers: 3
Failed Answers: 0
Average Response Time: 1.95 seconds
Success Rate: 100.0%


## Interactive Greek Derby Chatbot with Memory


In [14]:
from typing import List, Dict, Any
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

class GreekDerbyChatbot:
    def __init__(self, rag_graph, llm):
        self.rag_graph = rag_graph
        self.llm = llm
        self.memory = ConversationBufferMemory(return_messages=True)
        self.conversation_history = []
        
        # Enhanced prompt for conversational RAG
        self.chat_prompt = ChatPromptTemplate.from_messages([
            ("system", """Είστε ένας εξειδικευμένος βοηθός για το ελληνικό ποδόσφαιρο και το ντέρμπι Ολυμπιακός-Παναθηναϊκός.
            
Χρησιμοποιήστε τις παρακάτω πληροφορίες για να απαντήσετε στην ερώτηση του χρήστη.
Αν δεν γνωρίζετε την απάντηση, πείτε ότι δεν γνωρίζετε.
Απαντήστε στα ελληνικά με φιλικό και ενημερωτικό τρόπο.
Κρατήστε τις απαντήσεις συνοπτικές αλλά πλήρεις.

Περιεχόμενο: {context}

Προηγούμενη συνομιλία:
{chat_history}

Ερώτηση: {question}
Απάντηση:"""),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}")
        ])
    
    def chat(self, user_input: str) -> str:
        """Process user input and return chatbot response"""
        try:
            # Get relevant context using RAG
            rag_response = self.rag_graph.invoke({"question": user_input})
            context = rag_response.get("context", [])
            
            # Format context for the prompt
            context_text = "\n\n".join([doc.page_content for doc in context])
            
            # Get conversation history
            chat_history = self.memory.chat_memory.messages
            
            # Create the prompt
            messages = self.chat_prompt.format_messages(
                context=context_text,
                chat_history=chat_history,
                question=user_input
            )
            
            # Get response from LLM
            response = self.llm.invoke(messages)
            
            # Store in memory
            self.memory.chat_memory.add_user_message(user_input)
            self.memory.chat_memory.add_ai_message(response.content)
            
            # Store in conversation history
            self.conversation_history.append({
                "user": user_input,
                "bot": response.content,
                "context_sources": [doc.metadata.get("source", "unknown") for doc in context]
            })
            
            return response.content
            
        except Exception as e:
            error_msg = f"Σφάλμα: {str(e)}"
            self.memory.chat_memory.add_user_message(user_input)
            self.memory.chat_memory.add_ai_message(error_msg)
            return error_msg
    
    def get_conversation_history(self) -> List[Dict[str, Any]]:
        """Get the full conversation history"""
        return self.conversation_history
    
    def clear_memory(self):
        """Clear conversation memory"""
        self.memory.clear()
        self.conversation_history = []
        print("Η μνήμη της συνομιλίας διαγράφηκε.")
    
    def get_memory_summary(self) -> str:
        """Get a summary of the conversation"""
        if not self.conversation_history:
            return "Δεν υπάρχει ιστορικό συνομιλίας."
        
        summary = f"Συνομιλία με {len(self.conversation_history)} ερωτήσεις:\n"
        for i, conv in enumerate(self.conversation_history, 1):
            summary += f"{i}. Ερώτηση: {conv['user'][:50]}...\n"
            summary += f"   Απάντηση: {conv['bot'][:50]}...\n"
        
        return summary

# Initialize the chatbot
chatbot = GreekDerbyChatbot(greek_derby_graph, llm)
print("🤖 Greek Derby Chatbot initialized with memory!")
print("You can now have a conversation about Olympiakos vs Panathinaikos derby.")


🤖 Greek Derby Chatbot initialized with memory!
You can now have a conversation about Olympiakos vs Panathinaikos derby.


C:\Users\PX\AppData\Local\Temp\ipykernel_10180\3506290698.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(return_messages=True)


In [15]:
# Interactive Chat Function
def interactive_chat():
    """Interactive chat function for the Greek Derby Chatbot"""
    print("=" * 60)
    print("🤖 GREEK DERBY CHATBOT - Interactive Mode")
    print("=" * 60)
    print("Type your questions in Greek about Olympiakos vs Panathinaikos derby")
    print("Commands:")
    print("  - Type 'quit' or 'exit' to end the conversation")
    print("  - Type 'history' to see conversation history")
    print("  - Type 'clear' to clear memory")
    print("  - Type 'help' for more commands")
    print("=" * 60)
    
    while True:
        try:
            # Get user input
            user_input = input("\n👤 You: ").strip()
            
            # Handle special commands
            if user_input.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Goodbye! Thanks for chatting about the Greek derby!")
                break
            elif user_input.lower() == 'history':
                print("\n📚 Conversation History:")
                print(chatbot.get_memory_summary())
                continue
            elif user_input.lower() == 'clear':
                chatbot.clear_memory()
                continue
            elif user_input.lower() == 'help':
                print("\n🆘 Available Commands:")
                print("  - Ask any question about Olympiakos vs Panathinaikos")
                print("  - 'history' - Show conversation history")
                print("  - 'clear' - Clear conversation memory")
                print("  - 'quit' or 'exit' - End conversation")
                continue
            elif not user_input:
                print("Please enter a question or command.")
                continue
            
            # Get chatbot response
            print("\n🤖 Bot: ", end="")
            response = chatbot.chat(user_input)
            print(response)
            
        except KeyboardInterrupt:
            print("\n\n👋 Goodbye! Thanks for chatting about the Greek derby!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again or type 'quit' to exit.")

# Quick chat function for single questions
def quick_chat(question: str):
    """Quick chat function for single questions"""
    print(f"👤 You: {question}")
    response = chatbot.chat(question)
    print(f"🤖 Bot: {response}")
    return response

# Example usage and demo
print("\n" + "="*50)
print("🚀 CHATBOT READY! Choose your interaction mode:")
print("="*50)
print("1. For interactive chat, run: interactive_chat()")
print("2. For quick questions, run: quick_chat('Your question in Greek')")
print("3. Example: quick_chat('Ποια είναι η ιστορία του ντέρμπι;')")
print("="*50)



🚀 CHATBOT READY! Choose your interaction mode:
1. For interactive chat, run: interactive_chat()
2. For quick questions, run: quick_chat('Your question in Greek')
3. Example: quick_chat('Ποια είναι η ιστορία του ντέρμπι;')


In [16]:
# Demo: Test the chatbot with sample questions
print("🎯 DEMO: Testing the Greek Derby Chatbot")
print("="*50)

# Test questions in Greek
demo_questions = [
    "Ποια είναι η ιστορία του ντέρμπι;",
    "Ποιος έχει κερδίσει περισσότερες φορές;",
    "Ποιοι είναι οι κορυφαίοι παίκτες;"
]

print("Testing with sample questions...\n")

for i, question in enumerate(demo_questions, 1):
    print(f"Demo Question {i}:")
    response = quick_chat(question)
    print("-" * 40)

print("\n✅ Demo completed! The chatbot is ready for interactive use.")
print("\n" + "="*60)
print("🚀 READY TO CHAT! Choose your option:")
print("="*60)
print("1. Run: interactive_chat()  # For full interactive mode")
print("2. Run: quick_chat('Your question')  # For single questions")
print("="*60)


🎯 DEMO: Testing the Greek Derby Chatbot
Testing with sample questions...

Demo Question 1:
👤 You: Ποια είναι η ιστορία του ντέρμπι;
🤖 Bot: Η ιστορία του ντέρμπι Ολυμπιακός - Παναθηναϊκός είναι πλούσια και γεμάτη πάθος. Πρόκειται για έναν από τους πιο σημαντικούς και ιστορικούς αγώνες του ελληνικού ποδοσφαίρου, που έχει ξεκινήσει από τη δεκαετία του 1920. Οι δύο ομάδες είναι από τις πιο επιτυχημένες στην Ελλάδα και το ντέρμπι αποτελεί σημείο αναφοράς για τους φιλάθλους τους.

Μέχρι σήμερα, οι αγώνες αυτοί έχουν χαρακτηριστεί από έντονο ανταγωνισμό, ιστορικά περιστατικά και πολλές ανατροπές. Το ντέρμπι δεν είναι μόνο αθλητική αναμέτρηση, αλλά και κοινωνικό φαινόμενο, με τους φιλάθλους να ζουν κάθε αγώνα με μεγάλη ένταση και αφοσίωση. 

Η κόντρα έχει αναδείξει πολλές σπουδαίες προσωπικότητες και στιγμές στον ελληνικό ποδόσφαιρο, με εξέχουσες νίκες και σημαντικούς τίτλους να κρίνεται σε αυτές τις αναμετρήσεις. Σήμερα, το ντέρμπι παραμένει μια από τις πιο αναμενόμενες πτυχές του ελληνικού π

## Advanced Chatbot Features


In [ ]:
# Advanced chatbot features
def get_chatbot_stats():
    """Get statistics about the chatbot conversation"""
    history = chatbot.get_conversation_history()
    if not history:
        return "No conversation yet."
    
    total_questions = len(history)
    total_chars = sum(len(conv['user']) + len(conv['bot']) for conv in history)
    avg_question_length = sum(len(conv['user']) for conv in history) / total_questions
    avg_answer_length = sum(len(conv['bot']) for conv in history) / total_questions
    
    return f"""
📊 Chatbot Statistics:
- Total Questions: {total_questions}
- Total Characters: {total_chars:,}
- Avg Question Length: {avg_question_length:.1f} chars
- Avg Answer Length: {avg_answer_length:.1f} chars
"""

def export_conversation(filename="greek_derby_chat.json"):
    """Export conversation to JSON file"""
    import json
    history = chatbot.get_conversation_history()
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(history, f, ensure_ascii=False, indent=2)
    print(f"Conversation exported to {filename}")

print("🔧 Advanced features available:")
print("- get_chatbot_stats() - View conversation statistics")
print("- export_conversation() - Export chat to JSON file")
print("- chatbot.clear_memory() - Clear conversation memory")
print("- chatbot.get_conversation_history() - Get full history")


🔧 Advanced features available:
- get_chatbot_stats() - View conversation statistics
- export_conversation() - Export chat to JSON file
- chatbot.clear_memory() - Clear conversation memory
- chatbot.get_conversation_history() - Get full history
